# Calculate Climatology for CESM1 S2S Runs

### CESM1 data is located in:


### Climatology is calculated following the SubX protocol. It is output to a file:


### Function for calculating climatology is located in:
`clim_utils.py`

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

from clim_utils import daily_climo_subx

### Pre-processing function called by `open_mfdataset` to handle files with missing data or incorrect times

In [2]:
# File Path information
model='CESM1_30LCAM5'
path='/glade/scratch/jrichter/CPC_DATA/'+model
dstr='00z_d01_d45'

# Variable to be processed
varname='pr'
lev='sfc'

# Years to be gotton
sdate='19990106' # This was set by hand as the first init date for the first year
#edate='20151231'
#sdate='20080106'
edate='20151231'
yrs_list=np.arange(1999,2016)

# Initialization months
mnums=['01','02','12']
mstrs=['jan','feb','dec']


# Ensemble members
enss=['00','01','02','03','04'] 

### Read in each ensemble member for hindcasts for 2m Temperature
* `init` is concat dimension for `open_mfdataset`
* Each month is read in individually, then all are combined by the `init` dimension
* `ens` for each `init` is read in, then the data are combined by the `ens` dimension

In [3]:
# Create empty list to append data for each ensemble member
fcst_ds_ens=[]

# Loop over ensembles
for iens in enss:

    # Create empty list to append data for each month
    fcst_ds_months=[]
    
    # Get list of files and read in data for each month for this ensemble member
    for mnum,mstr in zip(mnums,mstrs):
        
        # Get all the filenames for this month for all years
        fnames = [f'{path}/{varname}/{year}/{mnum}/{varname}_{lev}_{model}_*{mstr}{year}_{dstr}_m{iens}.nc' for year in yrs_list]       
        
        # Create list of all filenames for this ensemble member, month, and all years
        files1=[]
        for files in fnames:
            f2=glob.glob(files)
            for f in f2:
                files1.append(f)
       
        print(files1)
        
        # Read in data concatentating over the init dimension
        fcst_ds_tmp=xr.open_mfdataset(files1,parallel=True,combine='nested',
                                      decode_times=False,concat_dim='init')

        
        # Create dates for the init dimension and assign them
        init_dates_all=pd.date_range(start=sdate,end=edate,freq='7D')
        djf_dates=init_dates_all[(init_dates_all.month==int(mnum))]
        print("TEST: ",djf_dates)
        fcst_ds_tmp['init']=djf_dates

        # Append the latest month to the list
        fcst_ds_months.append(fcst_ds_tmp)
        
    
    # Combine the months into the init dimension
    fcst_ds_months = xr.combine_nested(fcst_ds_months, concat_dim=['init'])
    
    # Append this ensenble member
    fcst_ds_ens.append(fcst_ds_months)
    
    
fcst_ds_ens   

['/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/1999/01/pr_sfc_CESM1_30LCAM5_06jan1999_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/1999/01/pr_sfc_CESM1_30LCAM5_27jan1999_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/1999/01/pr_sfc_CESM1_30LCAM5_13jan1999_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/1999/01/pr_sfc_CESM1_30LCAM5_20jan1999_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/2000/01/pr_sfc_CESM1_30LCAM5_05jan2000_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/2000/01/pr_sfc_CESM1_30LCAM5_26jan2000_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/2000/01/pr_sfc_CESM1_30LCAM5_19jan2000_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/2000/01/pr_sfc_CESM1_30LCAM5_12jan2000_00z_d01_d45_m00.nc', '/glade/scratch/jrichter/CPC_DATA/CESM1_30LCAM5/pr/2001/01/pr_sfc_CESM1_30LCAM5_17jan2001_00z_d01_d45_m00.nc', 

[<xarray.Dataset>
 Dimensions:  (LAT: 181, LON: 360, TIME: 45, init: 220)
 Coordinates:
   * TIME     (TIME) float32 0.5 1.5 2.5 3.5 4.5 5.5 ... 40.5 41.5 42.5 43.5 44.5
   * LON      (LON) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
   * LAT      (LAT) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
   * init     (init) datetime64[ns] 1999-01-06 1999-01-13 ... 2015-12-30
 Data variables:
     PR       (init, TIME, LAT, LON) float32 dask.array<chunksize=(1, 45, 181, 360), meta=np.ndarray>
 Attributes:
     institution_id:  NCAR and NOAA/ESRL/PSD
     institution:     National Center for Atmospheric Research and NOAA Earth ...
     experiment_id:   Wed Nov 29 12:54:30 MST 2017
     model_id:        CESM1_30LCAM5
     frequency:       daily
     project_id:      SUBx project
     experiment:      1999-01-06 IC Sub-seasonal Forecast
     modeling_realm:  atmos
     realization:     000
     contact:         Jadwiga (Yaga) Richter (jrichter@ucar.edu), Judith

In [4]:
# Combine data over ensemble dimension
fcst_ds_ens = xr.combine_nested(fcst_ds_ens, concat_dim=['ens'])
fcst_ds_ens['ens']=np.arange(0,len(enss))

fcst_ds_ens

<xarray.Dataset>
Dimensions:  (LAT: 181, LON: 360, TIME: 45, ens: 5, init: 220)
Coordinates:
  * TIME     (TIME) float32 0.5 1.5 2.5 3.5 4.5 5.5 ... 40.5 41.5 42.5 43.5 44.5
  * LON      (LON) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * init     (init) datetime64[ns] 1999-01-06 1999-01-13 ... 2015-12-30
  * LAT      (LAT) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * ens      (ens) int64 0 1 2 3 4
Data variables:
    PR       (ens, init, TIME, LAT, LON) float32 dask.array<chunksize=(1, 1, 45, 181, 360), meta=np.ndarray>
Attributes:
    institution_id:  NCAR and NOAA/ESRL/PSD
    institution:     National Center for Atmospheric Research and NOAA Earth ...
    experiment_id:   Wed Nov 29 12:54:30 MST 2017
    model_id:        CESM1_30LCAM5
    frequency:       daily
    project_id:      SUBx project
    experiment:      1999-01-06 IC Sub-seasonal Forecast
    modeling_realm:  atmos
    realization:     000
    contact:         Jadwiga (Yaga) Richter (jrichter@ucar.edu), Judith Perlw...
    startyear:       1999
    startmonth:      01
    startday:        06
    Conventions:     CF-1.0
    Generator:       NCL v.6.4.0
    References:      Original post-processing scripts come from Dughong Min (...

### Calculate the Ensemble Mean

In [5]:
fcst_ds=fcst_ds_ens.mean(dim='ens')

In [6]:
fcst_ds

<xarray.Dataset>
Dimensions:  (LAT: 181, LON: 360, TIME: 45, init: 220)
Coordinates:
  * TIME     (TIME) float32 0.5 1.5 2.5 3.5 4.5 5.5 ... 40.5 41.5 42.5 43.5 44.5
  * LON      (LON) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * init     (init) datetime64[ns] 1999-01-06 1999-01-13 ... 2015-12-30
  * LAT      (LAT) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
Data variables:
    PR       (init, TIME, LAT, LON) float32 dask.array<chunksize=(1, 45, 181, 360), meta=np.ndarray>

### Determine leads and set them as integers for the lead dimension

In [7]:
nt=fcst_ds['TIME'].size
leads=np.arange(0,nt)
fcst_ds=fcst_ds.rename({'TIME':'lead','LAT':'lat','LON':'lon','PR':varname})
fcst_ds['lead']=leads

### Calculate the climatology and save to file

In [8]:
climo_out_path='/glade/scratch/kpegion/ESPWG/data/'+model+'/hcst/climo/'
cfname=climo_out_path+'climo_'+varname+'.nc'
print(cfname)
climo=daily_climo_subx(fcst_ds[varname],varname,fname=cfname)

/glade/scratch/kpegion/ESPWG/data/CESM1_30LCAM5/hcst/climo/climo_pr.nc


/glade/u/home/kpegion/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
